In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import gdown

%matplotlib inline

In [ ]:
df = pd.read_excel('Linkedin_cleaned_latest_mapped_cols.xlsx')

In [ ]:
titlecols = [col for col in df.columns if 'title' in col]
df['all_titles'] = df[titlecols].apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)#--------------------------------------remove punctuvation
df.drop(columns=titlecols, inplace=True)

org_cols = ([col for col in df.columns if 'org' in col])
df['all_org_cols'] = df[org_cols].apply(lambda row: ','.join(row.dropna().astype(str)), axis=1)
df.drop(columns=org_cols, inplace=True)

job_location_cols = [col for col in df.columns if 'location' in col]
df['all_loc_cols'] = df[job_location_cols].apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
df.drop(columns=job_location_cols, inplace=True)

institute_cols = [col for col in df.columns if 'institute' in col]
df['all_instutes'] = df[institute_cols].apply(lambda row: ','.join(row.dropna().astype(str)), axis=1)
df.drop(columns=institute_cols, inplace=True)

degree_cols = [col for col in df.columns if 'digree' in col and 'duration' not in col]
df['all_degree'] = df[degree_cols].apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
df.drop(columns=degree_cols, inplace=True)

# 'skill',
# 'company_emp_count', 'company_industry',

skil_col = [col for col in df.columns if 'skill' in col]
# df['all_skills'] = df[skil_col].apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
df.drop(columns=skil_col, inplace=True)

comp_emp_count = [col for col in df.columns if 'company' in col and 'emp' in col]            # how big are their company exp's so we are totaling the info.
# comp_emp_count.append('employee_count')
df['all_comp_emp_count'] = df[comp_emp_count].sum(axis=1)
df.drop(columns=comp_emp_count, inplace=True) #not enough data

comp_ind = [col for col in df.columns if 'company' in col and 'industry' in col]
df['all_comp_ind'] = df[comp_ind].apply(lambda row: ' '.join(row.dropna().astype(str)), axis=1)
df.drop(columns=comp_ind, inplace=True)

In [ ]:
df.to_csv('retrival_database.csv',index = False)

In [ ]:
del df['name']

In [ ]:
df = pd.get_dummies(df,columns = job_duration_cols)
df = pd.get_dummies(df,columns = degree_duration_cols)

In [ ]:
df.iloc[:,75:]

# vectorizing

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

def vectorize_columns_method1(df, columns_to_vectorize):
    tfidf_vectorizer_method1 = TfidfVectorizer(tokenizer=lambda x: x.split(','))
    job_titles_method1 = df[columns_to_vectorize].apply(lambda row: ','.join(row), axis=1)
    job_title_matrix_method1 = tfidf_vectorizer_method1.fit_transform(job_titles_method1)
    job_title_vector_df_method1 = pd.DataFrame(job_title_matrix_method1.toarray(), columns=tfidf_vectorizer_method1.get_feature_names_out())
    return job_title_vector_df_method1,tfidf_vectorizer_method1

def vectorize_columns_method2(df, columns_to_vectorize):
    tfidf_vectorizer_method2 = TfidfVectorizer()
    job_titles_method2 = df[columns_to_vectorize].apply(lambda row: ' '.join(row), axis=1)
    job_title_matrix_method2 = tfidf_vectorizer_method2.fit_transform(job_titles_method2)
    job_title_vector_df_method2 = pd.DataFrame(job_title_matrix_method2.toarray(), columns=tfidf_vectorizer_method2.get_feature_names_out())
    return job_title_vector_df_method2,tfidf_vectorizer_method2

# 'all_titles', 'all_org_cols', 'all_loc_cols', 'all_instutes',
#        'all_degree', 'all_comp_emp_count', 'all_comp_ind'

columns_to_vectorize_1 = ['all_org_cols','all_instutes']
columns_to_vectorize_2 = [ 'all_titles', 'all_loc_cols', 'all_degree', 'all_comp_ind']

vectorized_df_method1,tfidf_vectorizer_method1 = vectorize_columns_method1(df, columns_to_vectorize_1)
vectorized_df_method2,tfidf_vectorizer_method2 = vectorize_columns_method2(df, columns_to_vectorize_2)

# Concatenate the vectorized DataFrames horizontally
result_df = pd.concat([df, vectorized_df_method1, vectorized_df_method2], axis=1)

# Drop the non-vectorized columns
result_df = result_df.drop(columns=columns_to_vectorize_1)
result_df = result_df.drop(columns=columns_to_vectorize_2)
# Print the resulting DataFrame

In [ ]:
orgdf

In [ ]:
orgdf.to_csv('retrivaldata.csv',index=False)

In [ ]:
info_df = result_df.count()
info_df = pd.DataFrame({'Column': info_df.index, 'Non-Null Count': info_df.values})
info_df = info_df.sort_values(by='Non-Null Count', ascending=False)
info_df = info_df.reset_index(drop = True)
info_df

In [ ]:
result_df.info()

# ML

In [ ]:
# # Step 1: Split the Data
# from sklearn.model_selection import train_test_split

# # Assuming 'tfidf_df' is your TF-IDF vectorized DataFrame
# x_train, x_test = train_test_split(tfidf_df, test_size=0.2, random_state=42)

# # Step 2: Train a Model (k-Nearest Neighbors)
# from sklearn.neighbors import NearestNeighbors

# # Create the k-NN model
# knn_model = NearestNeighbors(n_neighbors=5, algorithm='auto', metric='cosine')
# knn_model.fit(x_train)

# # Step 3: Generate Recommendations
# # Assuming 'user_input' is the vectorized input of the user
# # Find the top 4 most similar profiles for each user input
# user_input = x_test.iloc[5]  # Change the index (5) to any user index you want to test
# user_input = user_input.values.reshape(1, -1)  # Reshape to 2D array

# # Find the top 4 most similar profiles for the specific user input
# distances, indices = knn_model.kneighbors(user_input, n_neighbors=4)

# # Get the indices of the most similar profiles for the specific user
# top_4_indices = indices[0]

# # Get the top 4 profiles for the specific user from the original DataFrame
# top_4_profiles = df.iloc[top_4_indices]


In [ ]:
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV

x_train, x_test = train_test_split(result_df, test_size=0.25, random_state=42)
from sklearn.neighbors import NearestNeighbors

knn_model = NearestNeighbors(algorithm='auto', metric='cosine')
param_grid = {'n_neighbors': [3, 5, 7, 10, 15, 20]}

grid_search = GridSearchCV(knn_model, param_grid, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(x_train)

best_k = grid_search.best_params_['n_neighbors']
print(best_k)

knn_model = NearestNeighbors(n_neighbors=best_k, algorithm='auto', metric='cosine')
knn_model.fit(x_train)

ValueError: ignored

In [ ]:
x_train[:1].to_csv('x_train.csv',index=False)

In [ ]:
x_train[:1]

In [ ]:
from google.colab import files
import pickle

with open('/content/knn_model_linkedin.pkl', 'wb') as file:
    pickle.dump(knn_model, file)

files.download('/content/knn_model_linkedin.pkl')

In [ ]:
user_input = x_test.iloc[5]  # Change the index (5) to any user index you want to test
user_input = user_input.values.reshape(1, -1)  # Reshape to 2D array

distances, indices = knn_model.kneighbors(user_input, n_neighbors=4)
print('distances ',distances)
top_4_indices = indices[0]

top_4_profiles = orgdf.iloc[top_4_indices]
top_4_profiles

In [ ]:
indices[0]

In [ ]:
distances

In [ ]:
cosine_distances = [0.00345241, 0.00346511, 0.00361493, 0.00381158]
cosine_similarity_scores = [1 - distance for distance in cosine_distances]

print("Cosine Similarity Scores:", cosine_similarity_scores)

In [ ]:
x_test

In [ ]:
user_input = x_test.iloc[10]
user_input = user_input.values.reshape(1, -1)  # Reshape to 2D array

# Find the top 4 most similar profiles for the specific user input
distances, indices = knn_model.kneighbors(user_input, n_neighbors=4)

# Get the indices of the most similar profiles for the specific user
top_4_indices = indices[0]

# Get the top 4 profiles for the specific user from the original DataFrame
top_4_profiles = orgdf.iloc[top_4_indices]

In [ ]:
top_4_profiles = top_4_profiles.dropna(axis=1, how='all')
top_4_profiles = top_4_profiles.loc[:, (top_4_profiles != 0.0).any()]
top_4_profiles

In [ ]:
row_index = 220
row_df = orgdf.iloc[row_index].to_frame().T
row_df = row_df.dropna(axis=1, how='all')
row_df = row_df.loc[:, (row_df != 0.0).any()]
row_df.to_csv('input.csv',index = False)

In [ ]:
x_train

In [ ]:
row_df

In [ ]:
distances

In [ ]:
top_4_profiles.to_csv('top_4_profiles.csv',index = False)